# TP4 : travail final sur l'évolution de l'image des femmes scientifiques francophones dans le corpus CAMille

## Constitution des critères du sous-corpus

In [ ]:
#import des librairies
import requests
from bs4 import BeautifulSoup

# URL de la page Wikipedia
url = "https://fr.wikipedia.org/wiki/Liste_de_femmes_scientifiques"

# Téléchargement du contenu de la page
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
#Sélection des scientifiques francophones
# A partir des drapeaux Belgique, France et Suisse
target_flags = [
    'Flag_of_Belgium.svg',
    'Flag_of_France.svg',
    'Flag_of_Switzerland.svg'
]

# Sélectionner les éléments <li> correspondant aux pays cibles
francophones = []
for li in soup.find_all('li'):
    flag_span = li.find('span', class_='flagicon')
    if flag_span and any(flag in flag_span.a['href'] for flag in target_flags):
        francophones.append(li)

# Afficher les éléments <li> correspondants
print(francophones[:10])

In [ ]:
#Sélection des francophones nées avant 1945
scientists = []
for li in francophones:
    link = li.find('a', href=True)
    if link:
        # Trouver le texte contenant la date de naissance
        text_parts = li.get_text().split('(')
        if len(text_parts) > 1:
            birth_year_text = text_parts[-1].split('-')[0]
            if birth_year_text.isdigit():
                birth_year = int(birth_year_text)
                if birth_year < 1945:
                    scientists.append(li)

# Contrôler les éléments <li> correspondants
for li_element in scientists:
    print(li_element.get_text())

In [ ]:
# Affichage du nombre de scientifiques identifiées
len(scientists)

In [ ]:
#Nettoyer la liste des scientifiques
for li in scientists:
    # Trouver toutes les balises 'a' à l'intérieur de chaque balise 'li'
    ancres = li.find_all('a', class_=False)
    for ancre in ancres :
        # Imprimer les titres à l'intérieur de chaque balise 'a'
        scientist_name = ancre.text
        # Ecarter les mots spécialités notées en minuscules
        if not scientist_name.islower() :
            print(scientist_name)


## Imports et données pour le sous-corpus

In [ ]:
# Import des librairies
from collections import defaultdict
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import nltk
#nltk.download('punkt')
import re
from nltk.tokenize import sent_tokenize
import spacy
from spacy.lang.fr.examples import sentences

nlp = spacy.load('fr_core_news_md')

In [ ]:
# répertoires
indir = "../data/tp4" # import txt de Camille
outdir = "../data/tp4/txt_clean" # fichiers txt nettoyés

# données
annee1 = 1870 # première année d'apparition
all_years = [str(year) for year in range(int(annee1), 1971)]
autrices =  {"Marie Curie", "Irène Joliot-Curie", "Sophie Germain", "Lucia de Brouckère"}
infile = "articles.txt" #tout le sous-corpus dans 1 seul fichier
outfile = "../data/tp4/sents.txt" #analyse de sentiments

In [ ]:
# Lister les fichiers des articles 
files = []
for f in os.listdir(indir):
    if os.path.isfile(os.path.join(indir, f)):
        files.append(f)
files[:10]

In [ ]:
# Compter le nombre de fichiers
len(files)

## Explorer le sous-corpus

### statistiques des journaux

In [ ]:
# Compter le nombre de journaux représentés
count_decade = defaultdict(int)
count_month = defaultdict(int)
count_newspapers = defaultdict(int)
covered_years = set()

for f in files:
    if "_" in f and f.endswith("txt"):
        elems = f.split("_")
        if len(elems) < 3:
            print(f"Anomalous file: {f}")
            continue
        newspaper = elems[1]

        year = elems[2].split("-")[0]
        covered_years.add(year)
        decade = year[:3] + "0s"
        month = int(elems[2].split("-")[1])
        
        count_decade[decade] += 1
        count_newspapers[newspaper] += 1
        count_month[month] += 1
    else:
        print(f"Anomalous file: {f}")

len(files)

In [ ]:
print('Cette sélection contient :')
print(f"{count_newspapers['JB421']} exemplaires de L'Avenir du Luxembourg")
print(f"{count_newspapers['JB427']} exemplaires de La Libre Belgique")
print(f"{count_newspapers['JB555']} exemplaires de L'Indépendance belge")
print(f"{count_newspapers['JB555A']} exemplaires de L'Indépendance belge (édité en Angleterre)")
print(f"{count_newspapers['JB567']} exemplaires du Journal de Bruxelles")
print(f"{count_newspapers['JB572']} exemplaires du Journal de Charleroi")
print(f"{count_newspapers['JB638']} exemplaires de La Meuse")
print(f"{count_newspapers['JB685']} exemplaires de Le Petit Bleu")
print(f"{count_newspapers['JB729']} exemplaires de Le Vingtième Siècle")
print(f"{count_newspapers['JB773']} exemplaires de Vers l'Avenir")
print(f"{count_newspapers['JB837']} exemplaires de Le Peuple")
print(f"{count_newspapers['JB838']} exemplaires du journal Le Soir")
print(f"{count_newspapers['JB1051']} exemplaires de Le Drapeau rouge")

In [ ]:
print(f"La répartition par décennie :")
print(f"{count_decade['1880s']} fichiers pour la décennie 1880s")
print(f"{count_decade['1890s']} fichiers pour la décennie 1890s")
print(f"{count_decade['1900s']} fichiers pour la décennie 1900s")
print(f"{count_decade['1910s']} fichiers pour la décennie 1910s")
print(f"{count_decade['1920s']} fichiers pour la décennie 1920s")
print(f"{count_decade['1930s']} fichiers pour la décennie 1930s")
print(f"{count_decade['1940s']} fichiers pour la décennie 1940s")
print(f"{count_decade['1950s']} fichiers pour la décennie 1950s")
print(f"{count_decade['1960s']} fichiers pour la décennie 1960s")
print(f"{count_decade['1970s']} fichiers pour la décennie 1970s")

In [ ]:
missing_years = [y for y in all_years if y not in covered_years]
print(f"Années manquantes: {', '.join(missing_years)}")

### nombre de citations par autrice

In [ ]:
# liste des citations par autrice par année
autrices_annees ={}

for autrice in autrices:
    autrices_annees[autrice]={}
    for annee in all_years:
        autrices_annees[autrice][annee] = {}
autrices_annees

In [ ]:
# comptage du nombre de citations par autrice par année
nb_articles = 0
total_articles = 0
no_annee = annee
for autrice in autrices :
    total_articles += nb_articles
    nb_articles = 0
    #print(autrice)
    for f in files:
        if "_" in f and f.endswith("txt") :
            #print(f)
            filepath = indir +"/" + f
            file = open(filepath,"r", encoding="utf-8")
            lines = file.readlines()
            elems = f.split("_")
            if len(elems) < 3:
                #print(f"Anomalous file: {f}")
                continue
            annee = elems[2].split("-")[0]
            #print(annee)
            for line in lines :
                if autrice in line:
                    #print(line[:100])
                    if annee>no_annee :
                        no_annee = annee
                    nb_articles +=1
                    #print(nb_articles)
                    autrices_annees[autrice][annee] = nb_articles
#print(total_articles)                    
#autrices_annees
print(len(files))

In [ ]:
autrices_annees

In [ ]:
# Liste pour formater les données
data = []

# Convertir les données en DataFrame pandas
for personne, donnees_annees in autrices_annees.items():
        for annee, nb_articles in donnees_annees.items():
                data.append({
                'Scientifiques' : personne,
                'Année' : annee,
                'Nombre d\'articles' : nb_articles
                })


In [ ]:
# Création d'un DataFrame pour pandas
df = pd.DataFrame(autrices_annees)
print(df)

In [ ]:
#corriger le type des données
for autrice in autrices :
    #df_sorted.fillna(autrice)
    df[autrice]=pd.to_numeric(df[autrice])
df.dtypes

In [ ]:
print(df.items())

In [ ]:
# création du graphique
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.heatmap(df, cmap='RdYlGn_r', linewidths=0.5, annot=False)

In [ ]:
df.plot()
plt.show()

## Analyser le sous-corpus

### Préparer le sous-corpus

In [ ]:
print(len(files))

In [ ]:
#nettoyer le sous-corpus
if not os.path.exists(outdir):
    os.mkdir(outdir)

#paramétrage des termes de recherche
query = ["Marie Curie","Irène Joliot-Curie","Sophie Germain","Lucia de Brouckère"]

# Création d'une regex afin de trouver les mots de la liste query dans le texte
regex = re.compile(f"\\b({'|'.join(query)})\\b", re.IGNORECASE)

#nettoyage et export dans le nouveau répertoire
for file in files:
    if file.endswith(".txt"):
        relevant_sentences = []
        f_in = open(os.path.join(indir, file), encoding="utf-8")
        text = f_in.read()
        for sentence in sent_tokenize(text):
            if regex.search(sentence):
                relevant_sentences.append(sentence)
        f_in.close()
        f_out = open(os.path.join(outdir, file), "w", encoding="utf-8")
        f_out.write("\n\n".join(relevant_sentences))
        f_out.close()

len(files)

In [ ]:
# Lister les fichiers des articles 
files = []
for f in os.listdir(outdir):
    if os.path.isfile(os.path.join(indir, f)):
        files.append(f)
print(len(files))

In [ ]:
# intégrer tous les fichiers dans un seul fichier texte
articles = ""

for f in files :
    if "_" in f and f.endswith("txt") :
            filepath = outdir +"/" + f
            file = open(filepath,"r", encoding="utf-8")
            lines = file.readlines()
            for line in lines:
                #print(line)
                articles += line
print(articles[:100])
print(len(articles))

In [ ]:
f_export = open(infile, "w",encoding="utf-8")
f_export.write(articles)
f_export.close

f_export = open(infile, "r",encoding="utf-8")
print(f_export.read())

### Entités nommées

In [ ]:
#charger le texte
text = open(infile, encoding='utf-8').read()

In [ ]:
%%time
# Traiter le texte
#nlp.max_length=len(articles)
doc = nlp(text)

In [ ]:
# Compter les entités
people = defaultdict(int)
place = defaultdict(int)
organization = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1
    else : 
        if ent.label_ == "LOC" and len(ent.text) > 3:
            place[ent.text] += 1
        else : 
            if ent.label_ == "ORG" and len(ent.text) > 3:
                organization[ent.text] += 1

In [ ]:
# Trier et imprimer
#people
sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)
print("Apparition des personnes dans le corpus")
for person, freq in sorted_people[:10]:
    print(f"{person} :{freq}")
#place
print("---")
sorted_place = sorted(place.items(), key=lambda kv: kv[1], reverse=True)
print("Apparition des localisations dans le corpus")
for place, freq in sorted_place[:10]:
    print(f"{place} : {freq}")
#organization
print("---")
print("Apparition des organisations dans le corpus")
sorted_organization = sorted(organization.items(), key=lambda kv: kv[1], reverse=True)
for organization, freq in sorted_organization[:10]:
    print(f"{organization} : {freq}")

### Analyse des sentiments

#### Préparation du sous-corpus

In [ ]:
# créer un modèle de dictionnaire
autrices_sentiments ={}

for autrice in autrices:
    autrices_sentiments[autrice]={}
    for annee in all_years:
        autrices_sentiments[autrice][annee] = ""

In [ ]:
autrices_sentiments

In [ ]:
# copier le modèle pour l'analyse des sentiments
autrices_analyse = autrices_sentiments 

In [ ]:
# remplissage des citations dans autrices_sentiments

liste_phrases=[]
for autrice in autrices :
    
    #print(autrice)
    for f in files:
        if "_" in f and f.endswith("txt") :
            filepath = outdir +"/" + f
            file = open(filepath,"r", encoding="utf-8")
            lines = file.readlines()
            elems = f.split("_")
            if len(elems) < 3:
                #print(f"Anomalous file: {f}")
                continue
            annee = elems[2].split("-")[0]
            citations = ""
            #print(citations)
            for line in lines :              
                #print(citations)
                if autrice in line:
                    print(f)
                    print(autrice)
                    print(annee)
                    print(line[:100])
                    liste_phrases.append(line)
                    autrices_sentiments[autrice][annee] = liste_phrases                                


In [ ]:
autrices_sentiments

#### Chargement du modèle

In [ ]:
# Use a pipeline as a high-level helper
#from transformers import pipeline

from transformers import AutoModelForSequenceClassification, BertForSequenceClassification
from transformers import (XLMRobertaConfig, XLMRobertaTokenizer, TFXLMRobertaModel)            
from transformers import AutoTokenizer, AutoConfig, TFAutoModel, TFAutoModelForSequenceClassification 
from transformers import pipeline

PRETRAINED_MODEL_TYPES = {
    'xlmroberta': (AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, "tblard/tf-allocine")
}

config_class, model_class, tokenizer_class, model_name = PRETRAINED_MODEL_TYPES['xlmroberta']

# Download vocabulary from huggingface.co and cache.
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine",use_fast=False) #fast tokenizer
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

sentiment_analyser = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

#### Analyser le sentiment des phrases

In [ ]:
# analyse de quelques années choisies
phrase1=autrices_sentiments['Marie Curie']['1934']
print(len(phrase1))

In [ ]:
senti1=sentiment_analyser(phrase1)

In [ ]:

yes = 0
for senti in senti1:
    if 'POSITIVE' in senti :
        nb_positif =+ 1
moy_positif=nb_positif/len(senti1)
print(moyenne_positif) 

In [ ]:
# analyse des phrases et stockage dans un dictionnaire
data_analyser = []
autrices={'Marie Curie'}
all_years2 = [str(year) for year in range(1933, 1935)]

for autrice in autrices:
    for annee in all_years:
        phrases = autrices_sentiments[autrice][annee]
        for phrase in phrases :
            #print(phrase)
            #print(len(phrase))
            liste_analyser = []
            if len(phrase) > 0 :
                try :
                    sentiment_analyser(phrase)
                except Exception as e:
                    print(f"Erreur lors de l'analyse de la phrase : {phrase}")
                    autrices_sentiments[autrice][annee]="" # si non analysable, phrase écartée du sous-corpus 
                    continue 
                phrase = autrices_sentiments[autrice][annee]
                if len(phrase) > 2 :
                    senti = sentiment_analyser(phrase)
                    liste_analyser = [autrice, annee, senti]
                    #print(liste_analyser)
                    data_analyser.append(liste_analyser)

In [ ]:
data_analyser

In [ ]:
data = data_analyser

In [ ]:
# Dictionnaire pour stocker les moyennes
moyennes = {}

for entry in data:
    autrice, annee, liste_sentiments = entry
    
    if autrice not in moyennes:
        moyennes[autrice] = {}
    
    if annee not in moyennes[autrice]:
        moyennes[autrice][annee] = {'nb_avis': 0, 'nb_neutralite': 0, 'total_avis': 0, 'total_neutralite': 0}

    #print(liste_sentiments)
    list_senti = liste_sentiments
    if isinstance(liste_sentiments, list) :
        liste_senti = liste_sentiments
        #print(liste_senti[:10])    
    #dict_sentiments = sentiments[0]
    #nb_sentiments = len(liste_sentiments)
    #print(nb_sentiments)
        for sentiment in list_senti:
            avis = sentiment['label']
            #print(avis)
            neutralite = sentiment['score']
        
        # Mettre à jour les totaux
            if avis == 'POSITIVE' or avis == 'NEGATIVE':  # Ignorer si le sentiment n'est pas positif ou négatif
                moyennes[autrice][annee]['total_avis'] += 1
                if avis == 'POSITIVE':
                    moyennes[autrice][annee]['nb_avis'] += 1  
                else :
                    moyennes[autrice][annee]['nb_avis'] +=0
        
        moyennes[autrice][annee]['total_neutralite'] += neutralite
        moyennes[autrice][annee]['nb_neutralite'] += 1
    else :
        if isinstance(liste_sentiments,int) :
            #liste_senti = str(liste_sentiments)
            #print(liste_senti[:10])
            continue
        else :
            print('list_senti chaisplusquoifaire')

# Calcul des moyennes
for autrice, annees in moyennes.items():
    for annee, valeurs in annees.items():
        moyennes[autrice][annee]['moyenne_avis'] = valeurs['nb_avis'] / valeurs['total_avis'] if valeurs['total_avis'] > 0 else 0
        moyennes[autrice][annee]['moyenne_neutralite'] = valeurs['total_neutralite'] / valeurs['nb_neutralite'] if valeurs['nb_neutralite'] > 0 else 0

# Affichage des moyennes
for autrice, annees in moyennes.items():
    for annee, valeurs in annees.items():
        print(f"{autrice} {annee}: Moyenne d'avis - {valeurs['moyenne_avis']}, Moyenne de neutralite - {valeurs['moyenne_neutralite']}")

In [ ]:
moyennes

In [ ]:
moyennes['Irène Joliot-Curie']['1934']["moyenne_avis"]

#### représentation graphique de l'évolution des avis

In [ ]:
# Liste pour formater les données des avis
data_avis = []


# Convertir les données en DataFrame pandas
for autrice, annees in moyennes.items():
    for annee, valeurs in annees.items():
            data_avis.append({
                'Scientifiques' : autrice,
                'Année' : annee,
                'Moyenne des avis' : valeurs['moyenne_avis']
                 })


In [ ]:
data_avis

In [ ]:
# Création d'un DataFrame pour pandas
df = pd.DataFrame(data_avis)
print(df)

In [ ]:
#corriger le type des données
for autrice in autrices :
    #df_sorted.fillna(autrice)
    df[autrice]=pd.to_numeric(df[autrice])
df.dtypes

In [ ]:
#corriger le type des données
for autrice in autrices :
    df_data_avis[autrice]=pd.to_numeric(df[autrice])
df.dtypes

In [ ]:
print(df_data_avis.items())

In [ ]:
# création du graphique
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.heatmap(df_data_avis, cmap='RdYlGn_r', linewidths=0.5, annot=False)

In [ ]:
df_data_avis.plot()
plt.show()